# EO Products as Input Data

To support Applications that required the input EO Products to be previously staged-in, the _Best Practice for Earth Observation Application Package_ recommends the usage of a STAC Catalog with STAC Item files as the format of the data manifest.

The command-line tool must have an argument that represents the path to the folder where the STAC Catalog file is located. The input products are defined by the STAC Catalog with one or more STAC Items (and associated STAC Assets) as input files for processing.

The Application is a wrapper command-line tool that reads the STAC Catalog, selects the input Item Assets href and executes another command-line tool taking as argument the asset href (i.e. path to local file).

Let's create a simple commnd line tool using `click` that accepts a folder as input to the command line argument `--input-item`

In [1]:
import os
import click
import pystac
from click.testing import CliRunner


In [2]:
@click.command(
    short_help="Crop",
    help="Crops an input product",
)
@click.option(
    "--input-item",
    "staged_data",
    help="staged STAC catalog",
    type=click.Path(exists=True),
    required=True,
)
def crop(staged_data):
    print(f"Reading STAC catalog found in {staged_data}")
    
    catalog = pystac.read_file(os.path.join(staged_data, "catalog.json"))
    item = next(catalog.get_items())
    
    print(item)
    for key, asset in item.get_assets():
        print(key, asset.href)

Let's print the command line tool help:

In [3]:

runner = CliRunner()
result = runner.invoke(crop, ['--help'])

print(result.output)

Usage: crop [OPTIONS]

  Crops an input product

Options:
  --input-item PATH  staged STAC catalog  [required]
  --help             Show this message and exit.



Now, stage a Landsat-9 product according to _Best Practice for Earth Observation Application Package_:

In [4]:
import pystac
import stac_asset
import asyncio
import os
import nest_asyncio
nest_asyncio.apply()

In [5]:
config = stac_asset.Config(warn=True)

async def stage(href: str):

    item = pystac.read_file(href)

    os.makedirs(item.id, exist_ok=True)
    cwd = os.getcwd()

    os.chdir(item.id)
    item = await stac_asset.download_item(item=item, directory=".", config=config)
    os.chdir(cwd)

    cat = pystac.Catalog(
        id="catalog",
        description=f"catalog with staged {item.id}",
        title=f"catalog with staged {item.id}",
    )
    cat.add_item(item)

    cat.normalize_hrefs("./")
    cat.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

    return cat


Stage a Landsat-9 scene from Microsoft Planetary Computer:

In [6]:
href = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC09_L2SP_042033_20231015_02_T1"

cat = asyncio.run(stage(href))

Inspect the staged STAC Catalog:

In [7]:
cat.describe()

* <Catalog id=catalog>
  * <Item id=LC09_L2SP_042033_20231015_02_T1>


Now run the `crop` command line tool with the staged STAC Catalog:

In [8]:
runner = CliRunner()
result = runner.invoke(crop, ['--input-item', "./"])

print(result.output)

Reading STAC catalog found in ./
<Item id=LC09_L2SP_042033_20231015_02_T1>



The command-line tool reads the input EO product from the assets of the items included in the STAC Catalog file (catalog.json) in the specified directory.

The STAC items can be selected by the tool according to their respective metadata (e.g. bands, format, time).

Inside each STAC Item feature there are the corresponding STAC Assets for the product files (e.g. bands). The STAC Asset contains a link to the file associated with the STAC Item that can be downloaded or streamed (e.g. data, metadata, thumbnails) and can contain additional metadata.

In [9]:
item = next(cat.get_items())

for key, asset in item.get_assets().items():
    print(key, asset.href, asset.media_type, asset.roles)

qa ./LC09_L2SP_042033_20231015_20231016_02_T1_ST_QA.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data']
ang ./LC09_L2SP_042033_20231015_20231016_02_T1_ANG.txt text/plain ['metadata']
red ./LC09_L2SP_042033_20231015_20231016_02_T1_SR_B4.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data', 'reflectance']
blue ./LC09_L2SP_042033_20231015_20231016_02_T1_SR_B2.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data', 'reflectance']
drad ./LC09_L2SP_042033_20231015_20231016_02_T1_ST_DRAD.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data']
emis ./LC09_L2SP_042033_20231015_20231016_02_T1_ST_EMIS.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data']
emsd ./LC09_L2SP_042033_20231015_20231016_02_T1_ST_EMSD.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data']
trad ./LC09_L2SP_042033_20231015_20231016_02_T1_ST_TRAD.TIF image/tiff; application=geotiff; profile=cloud-optimized ['data']
urad ./LC09_L2SP_0